In [1]:
from PIL import Image

import gzip
import pickle as cPickle
import os
#os.system("cd input_signs")
num_records_input = 200
vector_size=256
#rotating signs 

name = "final_processed_data/S_"
new_name="final_rotated_signs/S_"
for m in range(num_records_input):

    f_name = name + str(m) + ".png"
    im = Image.open(f_name).convert('L')

    for n in range(10):
        rotate_val = (n*0.5)-2.5
        im1 = im.rotate(rotate_val, fillcolor=255)
        new_fname =  new_name + str(m) + "_" + str(n) + ".jpg"
        im1.save(new_fname)

print ("save completed")


save completed


In [4]:
#This program takes the 2000 input signature files as input
#Creates a serialized file along with the classification label
# Produces the output img2000.pkl.gz
# Generated file will be used to train CNN based feature extractor

from PIL import Image

import gzip
import pickle as cPickle

name = "final_rotated_signs/S_"
write_recs = []
for m in range(num_records_input):
    for n in range(10):
        file_name = name + str(m) + "_" + str(n) + ".jpg"
        im = Image.open(file_name)

        w,h =im.size

        #print ("w and h" , w, h)
        row1 = []
        col1 = []
        for i in range(w):
            col1 =[]
            for j in range(h):
                k=255-im.getpixel((i,j))
                col1.append(k)
            row1.append(col1)
            
        #if(m==0 and n==0):
            #print('len row1',len(row1))
            #print('row1',(row1))
        write_rec = [row1,m]
        write_recs.append(write_rec)


f = gzip.open('img2000.pkl.gz', 'wb')
cPickle.dump (write_recs, f)
f.close()


In [5]:
## This program takes img2000.pkl.gz file containing 2000 images and corresponding 200 classes as input.
## This program builds and trains a CNN to correctly classify these images into 200 classes
## Last layer consisting of 256 features will be used as feature vector for the signature images
## This feature vector will be used for signature comparision Neural Network / SVM classifier

import numpy as np
import random
from random import shuffle
import tensorflow.compat.v1 as tf
from numpy import genfromtxt
from numpy import recfromcsv
import csv
import pickle as cPickle
import gzip

# Read Signature image File consisting of 500 generated signatures

tf.disable_v2_behavior()

f = gzip.open('img2000.pkl.gz', 'rb')
 
inp_data = cPickle.load(f)
f.close()

#inp_data = inp_data[:500]
# Shuffle the records and fetch signatures and output classes into separate tensors

random.shuffle(inp_data)
inp_data = list(zip(*inp_data))
sig_data = inp_data[0]
outp_class = inp_data[1]

# Static Variables

NUM_EXAMPLES = num_records_input*7   # records used for training feature extractor
#NUM_RECORDS = 1
#NUM_HIDDEN = 30
#BETA = 0.01
LEARNING_RATE = 0.5
batch_size = 20
epoch=25

# Separate training and validation data

val_input = sig_data[NUM_EXAMPLES:]
val_output = outp_class[NUM_EXAMPLES:]
train_input = sig_data[:NUM_EXAMPLES]
train_output = outp_class[:NUM_EXAMPLES]

print ("training and validation data loaded")

print ("length of train input",len(train_input))
print ("length of train output",len(train_output))
print ("length of validation input",len(val_input))
print ("length of validation output",len(val_output))

#Building the TensorFlow Computational Graph


#Placeholders for signature data and output class
data = tf.placeholder(tf.float32, [None, 250,100]) 
test_data = tf.placeholder(tf.float32, [250,100]) 
target = tf.placeholder(tf.int32, [None])

# Convolution Layer 1 weights and biases
w_conv1 = tf.Variable(tf.truncated_normal([10, 10, 1, 4], stddev=0.1))
b_conv1 = tf.Variable(tf.constant(0.1, shape=[4]))

#Reshape the image into 4D

x_image = tf.reshape(data, [-1,250,100,1])

#Layer 1 Convolutions and Pooling
h_conv1 = tf.nn.conv2d(x_image, w_conv1, strides=[1,1,1,1], padding ='SAME') + b_conv1
h_relu1 = tf.nn.relu(h_conv1)
h_pool1 = tf.nn.max_pool(h_relu1,ksize=[1,2,2,1],strides =[1,2,2,1],padding='SAME')


#Convolution Layer 2 weights and biases
w_conv2 = tf.Variable(tf.truncated_normal([5, 5, 4, 8], stddev=0.1)) 
b_conv2 = tf.Variable(tf.constant(0.1, shape=[8]))

#Layer 2 Convolutions and Pooling
h_conv2 = tf.nn.conv2d(h_pool1, w_conv2, strides=[1,1,1,1], padding ='SAME') + b_conv2
h_relu2 = tf.nn.relu(h_conv2)
h_pool2 = tf.nn.max_pool(h_relu2,ksize=[1,2,2,1],strides =[1,2,2,1],padding='SAME')

# More Convolutions will be added as required

# Dense Layer 1
w_dense1 = tf.Variable(tf.truncated_normal([63 * 25 * 8, vector_size], stddev=0.1)) 
b_dense1 = tf.Variable(tf.constant(0.1, shape=[vector_size]))

h_pool2_dense = tf.reshape(h_pool2, [-1, 63*25*8])
h_dense1 = tf.nn.relu(tf.matmul(h_pool2_dense, w_dense1) + b_dense1)

#droput after Layer 1

keep_prob = tf.placeholder(tf.float32)
h_dense1_drop = tf.nn.dropout(h_dense1, keep_prob)

# Dense Layer 2

w_dense2 = tf.Variable(tf.truncated_normal([vector_size, num_records_input], stddev=0.1))
b_dense2 = tf.Variable(tf.constant(0.1, shape=[num_records_input]))

# Result of the network
conv_rslt = tf.matmul(h_dense1_drop, w_dense2) + b_dense2

target1 = tf.one_hot(target,num_records_input)

#Cross Entropy Cost

cross_entropy = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=target1, logits=conv_rslt))

#Optimizer

optimizer = tf.train.AdamOptimizer(1e-4).minimize(cross_entropy)

# Computation of accuracy and error rate
incorrect_prediction = tf.not_equal(tf.argmax(conv_rslt,1),tf.argmax(target1,1))

error_rate = tf.reduce_mean(tf.cast(incorrect_prediction, tf.float32))


# Execution of Computational Graph

init_op = tf.global_variables_initializer()
sess = tf.Session()
sess.run(init_op)

# comput number of mini batches

no_of_batches = int(len(train_input) / batch_size)
print ("number of batches", no_of_batches)

# Train over number of epochs

for i in range(epoch):
    ptr = 0
    # Train the network for every mini batch
    #for j in range(no_of_batches):
    #for j in range(1):
    for j in range(no_of_batches):
        print("epoch",i,"batch",j)
#        print ("batch",j)
        inp, out = train_input[ptr:ptr+batch_size], train_output[ptr:ptr+batch_size]
        ptr+=batch_size
        #print(inp)
        #print(out)
        print("before optimizer")
        print("length of input and output",len(inp),len(out))
        sess.run(optimizer,{data: inp, target: out, keep_prob:0.8})
        print("after optimizer")
        
    # Print the train and validation error after each epoch
    print ("Epoch ",str(i))
    incorrect = sess.run(error_rate,{data: train_input, target: train_output, keep_prob:1})
    print('Epoch {:2d} Training error {:3.1f}%'.format(i + 1, 100 * incorrect))

    incorrect = sess.run(error_rate,{data: val_input, target: val_output, keep_prob:1})
    print('Epoch {:2d} Validation error {:3.1f}%'.format(i + 1, 100 * incorrect))


#sess.close()




Instructions for updating:
non-resource variables are not supported in the long term
training and validation data loaded
length of train input 1400
length of train output 1400
length of validation input 600
length of validation output 600
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See `tf.nn.softmax_cross_entropy_with_logits_v2`.

number of batches 70
epoch 0 batch 0
before optimizer
length of input and output 20 20
after optimizer
epoch 0 batch 1
before optimizer
length of input and output 20 20
after optimizer
epoch 0 batch 2
before optimizer
length of input and output 20 20
after optimizer
epoch 0 batch 3
before optimizer
length of input and output 20 20
after optimizer
epoch 0 batch 4
before optimizer
length of input and output 20 20
after optimizer
epoch 0 batch 5
before o

after optimizer
epoch 1 batch 13
before optimizer
length of input and output 20 20
after optimizer
epoch 1 batch 14
before optimizer
length of input and output 20 20
after optimizer
epoch 1 batch 15
before optimizer
length of input and output 20 20
after optimizer
epoch 1 batch 16
before optimizer
length of input and output 20 20
after optimizer
epoch 1 batch 17
before optimizer
length of input and output 20 20
after optimizer
epoch 1 batch 18
before optimizer
length of input and output 20 20
after optimizer
epoch 1 batch 19
before optimizer
length of input and output 20 20
after optimizer
epoch 1 batch 20
before optimizer
length of input and output 20 20
after optimizer
epoch 1 batch 21
before optimizer
length of input and output 20 20
after optimizer
epoch 1 batch 22
before optimizer
length of input and output 20 20
after optimizer
epoch 1 batch 23
before optimizer
length of input and output 20 20
after optimizer
epoch 1 batch 24
before optimizer
length of input and output 20 20
afte

after optimizer
epoch 2 batch 41
before optimizer
length of input and output 20 20
after optimizer
epoch 2 batch 42
before optimizer
length of input and output 20 20
after optimizer
epoch 2 batch 43
before optimizer
length of input and output 20 20
after optimizer
epoch 2 batch 44
before optimizer
length of input and output 20 20
after optimizer
epoch 2 batch 45
before optimizer
length of input and output 20 20
after optimizer
epoch 2 batch 46
before optimizer
length of input and output 20 20
after optimizer
epoch 2 batch 47
before optimizer
length of input and output 20 20
after optimizer
epoch 2 batch 48
before optimizer
length of input and output 20 20
after optimizer
epoch 2 batch 49
before optimizer
length of input and output 20 20
after optimizer
epoch 2 batch 50
before optimizer
length of input and output 20 20
after optimizer
epoch 2 batch 51
before optimizer
length of input and output 20 20
after optimizer
epoch 2 batch 52
before optimizer
length of input and output 20 20
afte

after optimizer
epoch 3 batch 69
before optimizer
length of input and output 20 20
after optimizer
Epoch  3
Epoch  4 Training error 43.3%
Epoch  4 Validation error 62.2%
epoch 4 batch 0
before optimizer
length of input and output 20 20
after optimizer
epoch 4 batch 1
before optimizer
length of input and output 20 20
after optimizer
epoch 4 batch 2
before optimizer
length of input and output 20 20
after optimizer
epoch 4 batch 3
before optimizer
length of input and output 20 20
after optimizer
epoch 4 batch 4
before optimizer
length of input and output 20 20
after optimizer
epoch 4 batch 5
before optimizer
length of input and output 20 20
after optimizer
epoch 4 batch 6
before optimizer
length of input and output 20 20
after optimizer
epoch 4 batch 7
before optimizer
length of input and output 20 20
after optimizer
epoch 4 batch 8
before optimizer
length of input and output 20 20
after optimizer
epoch 4 batch 9
before optimizer
length of input and output 20 20
after optimizer
epoch 4 ba

after optimizer
epoch 5 batch 27
before optimizer
length of input and output 20 20
after optimizer
epoch 5 batch 28
before optimizer
length of input and output 20 20
after optimizer
epoch 5 batch 29
before optimizer
length of input and output 20 20
after optimizer
epoch 5 batch 30
before optimizer
length of input and output 20 20
after optimizer
epoch 5 batch 31
before optimizer
length of input and output 20 20
after optimizer
epoch 5 batch 32
before optimizer
length of input and output 20 20
after optimizer
epoch 5 batch 33
before optimizer
length of input and output 20 20
after optimizer
epoch 5 batch 34
before optimizer
length of input and output 20 20
after optimizer
epoch 5 batch 35
before optimizer
length of input and output 20 20
after optimizer
epoch 5 batch 36
before optimizer
length of input and output 20 20
after optimizer
epoch 5 batch 37
before optimizer
length of input and output 20 20
after optimizer
epoch 5 batch 38
before optimizer
length of input and output 20 20
afte

...

after optimizer
epoch 19 batch 21
before optimizer
length of input and output 20 20
after optimizer
epoch 19 batch 22
before optimizer
length of input and output 20 20
after optimizer
epoch 19 batch 23
before optimizer
length of input and output 20 20
after optimizer
epoch 19 batch 24
before optimizer
length of input and output 20 20
after optimizer
epoch 19 batch 25
before optimizer
length of input and output 20 20
after optimizer
epoch 19 batch 26
before optimizer
length of input and output 20 20
after optimizer
epoch 19 batch 27
before optimizer
length of input and output 20 20
after optimizer
epoch 19 batch 28
before optimizer
length of input and output 20 20
after optimizer
epoch 19 batch 29
before optimizer
length of input and output 20 20
after optimizer
epoch 19 batch 30
before optimizer
length of input and output 20 20
after optimizer
epoch 19 batch 31
before optimizer
length of input and output 20 20
after optimizer
epoch 19 batch 32
before optimizer
length of input and outpu

after optimizer
epoch 20 batch 48
before optimizer
length of input and output 20 20
after optimizer
epoch 20 batch 49
before optimizer
length of input and output 20 20
after optimizer
epoch 20 batch 50
before optimizer
length of input and output 20 20
after optimizer
epoch 20 batch 51
before optimizer
length of input and output 20 20
after optimizer
epoch 20 batch 52
before optimizer
length of input and output 20 20
after optimizer
epoch 20 batch 53
before optimizer
length of input and output 20 20
after optimizer
epoch 20 batch 54
before optimizer
length of input and output 20 20
after optimizer
epoch 20 batch 55
before optimizer
length of input and output 20 20
after optimizer
epoch 20 batch 56
before optimizer
length of input and output 20 20
after optimizer
epoch 20 batch 57
before optimizer
length of input and output 20 20
after optimizer
epoch 20 batch 58
before optimizer
length of input and output 20 20
after optimizer
epoch 20 batch 59
before optimizer
length of input and outpu

after optimizer
epoch 22 batch 5
before optimizer
length of input and output 20 20
after optimizer
epoch 22 batch 6
before optimizer
length of input and output 20 20
after optimizer
epoch 22 batch 7
before optimizer
length of input and output 20 20
after optimizer
epoch 22 batch 8
before optimizer
length of input and output 20 20
after optimizer
epoch 22 batch 9
before optimizer
length of input and output 20 20
after optimizer
epoch 22 batch 10
before optimizer
length of input and output 20 20
after optimizer
epoch 22 batch 11
before optimizer
length of input and output 20 20
after optimizer
epoch 22 batch 12
before optimizer
length of input and output 20 20
after optimizer
epoch 22 batch 13
before optimizer
length of input and output 20 20
after optimizer
epoch 22 batch 14
before optimizer
length of input and output 20 20
after optimizer
epoch 22 batch 15
before optimizer
length of input and output 20 20
after optimizer
epoch 22 batch 16
before optimizer
length of input and output 20 

after optimizer
epoch 23 batch 32
before optimizer
length of input and output 20 20
after optimizer
epoch 23 batch 33
before optimizer
length of input and output 20 20
after optimizer
epoch 23 batch 34
before optimizer
length of input and output 20 20
after optimizer
epoch 23 batch 35
before optimizer
length of input and output 20 20
after optimizer
epoch 23 batch 36
before optimizer
length of input and output 20 20
after optimizer
epoch 23 batch 37
before optimizer
length of input and output 20 20
after optimizer
epoch 23 batch 38
before optimizer
length of input and output 20 20
after optimizer
epoch 23 batch 39
before optimizer
length of input and output 20 20
after optimizer
epoch 23 batch 40
before optimizer
length of input and output 20 20
after optimizer
epoch 23 batch 41
before optimizer
length of input and output 20 20
after optimizer
epoch 23 batch 42
before optimizer
length of input and output 20 20
after optimizer
epoch 23 batch 43
before optimizer
length of input and outpu

after optimizer
epoch 24 batch 59
before optimizer
length of input and output 20 20
after optimizer
epoch 24 batch 60
before optimizer
length of input and output 20 20
after optimizer
epoch 24 batch 61
before optimizer
length of input and output 20 20
after optimizer
epoch 24 batch 62
before optimizer
length of input and output 20 20
after optimizer
epoch 24 batch 63
before optimizer
length of input and output 20 20
after optimizer
epoch 24 batch 64
before optimizer
length of input and output 20 20
after optimizer
epoch 24 batch 65
before optimizer
length of input and output 20 20
after optimizer
epoch 24 batch 66
before optimizer
length of input and output 20 20
after optimizer
epoch 24 batch 67
before optimizer
length of input and output 20 20
after optimizer
epoch 24 batch 68
before optimizer
length of input and output 20 20
after optimizer
epoch 24 batch 69
before optimizer
length of input and output 20 20
after optimizer
Epoch  24
Epoch 25 Training error 0.1%
Epoch 25 Validation e

In [66]:
im = Image.open("temp.png").convert('L')
im2 = Image.open("temp4.png").convert('L')
#img = np.array(im).reshape(1,250,100)
w,h =im.size
w2,h2 =im2.size

print ("w and h" , w, h)
print ("w and h" , w2, h2)

row1 = []
row2 = []
col1 = []
col2 = []

for i in range(250):
    col1 =[]
    col2 =[]
    for j in range(100):
        #print(im.getpixel((i,j)))
        k=255-im.getpixel((i,j))
        k2=255-im2.getpixel((i,j))
        col1.append(k)
        col2.append(k2)
    row1.append(col1)
    row2.append(col2)
inp=row1
inp2=row2

w and h 250 100
w and h 250 100


In [67]:
inp= np.array(inp).reshape(1,250,100)
inp2= np.array(inp2).reshape(1,250,100)

temp_output = sess.run(h_dense1,{data: inp, keep_prob:1})
temp_output2 = sess.run(h_dense1,{data: inp2, keep_prob:1})

In [68]:
print(len(temp_output[0]))
print(temp_output)

256
[[ 0.        21.994263   0.         0.         0.         0.
   0.         0.         0.         0.         0.         0.
   0.         0.         0.         0.         0.         0.
   0.         0.         0.         0.         0.         0.
   0.         0.         9.160078   0.         0.         0.
   0.         0.         0.         0.         0.         0.
   0.         0.         0.         0.         0.         0.
   0.        70.44724    0.         0.        16.688951   0.
   0.         0.         0.         0.         0.         0.
   0.         0.         0.         0.         0.         0.
   0.         0.         0.         0.        15.223642   0.
   0.         0.         0.         0.         9.503611   0.
   0.         0.         0.         0.         0.         4.3265605
   0.         0.         0.         4.6916885  0.         0.
   0.         0.         0.         0.         0.         0.
   0.         0.         0.         0.         0.         0.
   0.        

In [69]:
print(len(temp_output2[0]))
print(temp_output2)

256
[[  0.          0.          0.          0.          0.         31.807339
    0.9135857   0.          0.          0.          0.          0.
    0.          0.          0.          0.          0.          0.
    0.          0.          0.          0.          0.          0.
    0.          0.          0.          0.          0.          3.607292
    0.          0.          0.          0.          0.          0.
    0.          0.          0.          0.          0.          0.
    0.          0.          0.          0.          0.          0.
    0.          0.          0.          0.          0.         20.631332
    0.          0.         41.799477    8.604192    0.          0.
    0.          0.          0.          0.          0.          0.
    0.          0.          0.          0.          0.          0.
    0.          0.          0.          0.          0.          0.
    0.          0.          0.          0.          0.          0.
    0.         40.42041     0.         3

In [70]:
#from scipy import spatial
from numpy import dot
from numpy.linalg import norm

result = dot(temp_output[0], temp_output2[0])/(norm(temp_output[0])*norm(temp_output2[0]))

dist = np.sqrt(((temp_output[0] - temp_output2[0]) ** 2).sum())

In [71]:
print(dist)

353.24524


In [72]:
if(dist<200):
    print("its a match")
else:
    print("Not matching")


Not matching
